# Read and Plot Data

In [1]:
import functionsMasterProjectMeinhart as fmpm
from scipy.signal import savgol_filter
from matplotlib.widgets import Button

%matplotlib auto

Using matplotlib backend: TkAgg


In [7]:
# select file (csv)
file_dir  = r'E:\Physio_Data\Subject_06'
file_name = 'subject06.csv'

# selecet time range [min:sec]
start_min_sec = '15:0'
stop_min_sec  = '31:0'



sampling_rate = 256 # [Hz]

# split time string and convert to float
start_min = float(start_min_sec.split(':')[0])
start_sec = float(start_min_sec.split(':')[1])
stop_min = float(stop_min_sec.split(':')[0])
stop_sec = float(stop_min_sec.split(':')[1])

# start and stop time in seconds
start_time = start_min*60 + start_sec # [s]
stop_time = stop_min*60 + stop_sec # [s]

# join data path
data_path = os.path.join(file_dir, file_name)

# get data from selected file
sensor_data = fmpm.get_sensor_data(in_file=data_path,
                                   sampling_rate=sampling_rate,
                                   start_time=start_time,
                                   stop_time=stop_time,
                                   add_info=file_name)

if sensor_data is not 0:
    
    fig, axs = plt.subplots(2,1, sharex=True, figsize=(18,9))
    plt.subplots_adjust(bottom=0.2) # make space for buttons
    
    
    # class for marking area via button        
    class MarkPlot:

        def new_plot(self):
            
            fig.suptitle(' ')
            
            plt.sca(axs[0])
            axs[0].clear()
            fmpm.plot_signal(sensor_data['Acc'],sensor_data['time'],
                        Title='Acceleration Profile')
            plt.gca().set_xlabel('')
            plt.sca(axs[1])
            axs[1].clear()
            fmpm.plot_signal(sensor_data['Gyr'],sensor_data['time'],
                        Title='Angular Velocity Profile',
                        xLabel=r'$time \enspace [s]$',
                        yLabel=r'$vel \enspace [\frac{deg}{s}]$')
            plt.draw()
        
        def unmark(self, event):
            self.new_plot()
        
        def select_mark_area(self, event):
            
            # self.new_plot()
            
            # select two points for time range
            selected_points = plt.ginput(2)

            # check if two points are selected
            if np.shape(selected_points)[0] is 2:

                # selected start and stop time
                start_time_sel = selected_points[0][0]
                stop_time_sel  = selected_points[1][0]

                # change start and stop time if start time is higher than stop time
                if start_time_sel > stop_time_sel:
                    puffer = start_time_sel
                    start_time_sel = stop_time_sel
                    stop_time_sel = puffer

                # calculate corresponding index   !!! minus general start time
                start_time_sel_index = int((start_time_sel-start_time)*sampling_rate)
                stop_time_sel_index  = int((stop_time_sel-start_time)*sampling_rate)

                # refresh title of figure and show selected time range
                fig.suptitle(r'Selected time range: {0:02d}:{1:05.2f} $\rightarrow$ {2:02d}:{3:05.2f}'.format(
                    int(start_time_sel/60),start_time_sel%60,int(stop_time_sel/60),stop_time_sel%60),fontsize=20)


                # mark selected area
                for ii in range(2):
                        plt.sca(axs[ii])
                        plt.axvspan(start_time_sel, stop_time_sel, color='y', alpha=0.3, lw=0)
                plt.draw()


    callback = MarkPlot()
    callback.new_plot()

    ax_unmark = plt.axes([0.57, 0.05, 0.15, 0.075])
    ax_select = plt.axes([0.74, 0.05, 0.15, 0.075])

    b_unmark = Button(ax_unmark, 'Unmark Area')
    b_unmark.on_clicked(callback.unmark)
    b_select = Button(ax_select, 'Select Time Range')
    b_select.on_clicked(callback.select_mark_area)
